In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier




/tmp/ipykernel_8763/3016242139.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
df=pd.read_csv("titanic dataset.csv")
df.tail()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
413,1305,0,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,0,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,0,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S
417,1309,0,3,"Peter, Master. Michael J",male,NaN,1,1,2668,22.3583,NaN,C


In [3]:
df.columns


Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [4]:
df.drop(columns= ['PassengerId','Name','Ticket','Cabin'],inplace=True)

In [5]:
df

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,34.5,0,0,7.8292,Q
1,1,3,female,47.0,1,0,7.0000,S
2,0,2,male,62.0,0,0,9.6875,Q
3,0,3,male,27.0,0,0,8.6625,S
4,1,3,female,22.0,1,1,12.2875,S
...,...,...,...,...,...,...,...,...
413,0,3,male,NaN,0,0,8.0500,S
414,1,1,female,39.0,0,0,108.9000,C
415,0,3,male,38.5,0,0,7.2500,S
416,0,3,male,NaN,0,0,8.0500,S


In [6]:
df.dtypes

Survived      int64
Pclass        int64
Sex          object
Age         float64
SibSp         int64
Parch         int64
Fare        float64
Embarked     object
dtype: object

In [7]:
df.isnull().sum()

Survived     0
Pclass       0
Sex          0
Age         86
SibSp        0
Parch        0
Fare         1
Embarked     0
dtype: int64

In [8]:
rows_with_s = df[df['Fare'].astype(str).str.contains('S', case=False, na=False)]
rows_with_s

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked


In [9]:
df_x = df.drop(columns=["Survived"])
df_y= df["Survived"]

In [10]:
x_train,x_test,y_train,y_test = train_test_split(df_x,df_y,test_size=0.2,random_state=42)
y_train

336    0
31     0
84     0
287    0
317    0
      ..
71     0
106    0
270    0
348    0
102    0
Name: Survived, Length: 334, dtype: int64

In [11]:
from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2 
from sklearn.pipeline import Pipeline, make_pipeline

using ColumnTransformer to fill missing val by Imputer.SimpleImputer() fills empty cols with mean value. If we want to fill it with most frequent value then we have to use SimpleImputer(strategy = "most_frequent")
If you want to fill missing values with the least frequent value in each column using SimpleImputer, you should use strategy="constant" along with the fill_value parameter set to the least frequent value.
SimpleImputer(strategy='constant', fill_value=np.nan)

In [12]:
si=SimpleImputer()
si_cols =[2,5]
tf1 = ColumnTransformer(transformers=[('age_imputer',si,si_cols)],remainder ='passthrough')

In [13]:
tf1

ColumnTransformer(remainder='passthrough',
                  transformers=[('age_imputer', SimpleImputer(), [2, 5])])

Now we will create transformer for one hot encoder

In [14]:
ohe = OneHotEncoder()
ohe_cols=[1,6]
tf2=ColumnTransformer(transformers=[('ohe-cat',
                          ohe,ohe_cols)],remainder='passthrough')

In [15]:
tf2

ColumnTransformer(remainder='passthrough',
                  transformers=[('ohe-cat', OneHotEncoder(), [1, 6])])

Creating transformer for Scaling purpose. Scaling should be done inorder to select features

In [16]:
clf = DecisionTreeClassifier()

In [17]:
# Now we will create a pipeline that  will wrap up all the above  predefined transformer
# the difference between Pipeline and make_pipeline is clearly visible in pipe1 and pipe2

In [18]:
pipe1 = Pipeline([('tf1',tf1),
                  ('tf2',tf2),
                 ('clf',clf)])
pipe1

Pipeline(steps=[('tf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('age_imputer',
                                                  SimpleImputer(), [2, 5])])),
                ('tf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe-cat', OneHotEncoder(),
                                                  [1, 6])])),
                ('clf', DecisionTreeClassifier())])

In [19]:
pipe2 = make_pipeline(tf1,tf2,tf3)

NameError: name 'tf3' is not defined

In [20]:
pipe1.fit(x_train,y_train)

ValueError: For a sparse output, all columns should be a numeric or convertible to a numeric.

In [ ]:
pipe1.predict(x_test)